#**Step 1: Setting Up YOLOv8**

YOLOv8 is provided by the Ultralytics library. Installing it is very straightforward.



In [1]:
# Install the Ultralytics library (includes YOLOv8)
!pip install ultralytics



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 16.5 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO

# pre-trained model
model = YOLO('yolov8n.pt')  # YOLOv8 Nano model

# test prediction on an example image
results = model.predict(source='https://ultralytics.com/images/zidane.jpg', save=True)
print("Detection completed successfully!")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 79.2MB/s]


100%|██████████| 49.2k/49.2k [00:00<00:00, 5.63MB/s]


Speed: 16.0ms preprocess, 306.2ms inference, 31.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
Detection completed successfully!


#**Step 2: Running YOLOv8 Pre-Trained Model**

YOLOv8 provides pre-trained models for detecting objects trained on the COCO dataset. These models can recognize 80 object classes such as people, cars, traffic lights, etc.



In [3]:
# YOLOv8 model (pre-trained on COCO dataset)
model = YOLO('yolov8n.pt')  # 'yolov8n' is the smallest version of YOLOv8

# Run inference on an example image
results = model.predict(source='https://ultralytics.com/images/zidane.jpg', save=True, conf=0.25)



Found https://ultralytics.com/images/zidane.jpg locally at zidane.jpg
image 1/1 /content/zidane.jpg: 384x640 2 persons, 1 tie, 151.0ms
Speed: 4.0ms preprocess, 151.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict2


In [4]:
import os
os.listdir('runs/detect/predict')


['zidane.jpg']

In [5]:
# Path to the image file in your Google Drive
image_path = "/content/drive/MyDrive/Objects detection /IMG_20241120_132353.jpg"

# Perform object detection
results = model.predict(source=image_path, save=True, conf=0.6)

print("Detection completed! Check the results in 'runs/detect/predict'.")





image 1/1 /content/drive/MyDrive/Objects detection /IMG_20241120_132353.jpg: 480x640 1 person, 5 cars, 320.6ms
Speed: 7.3ms preprocess, 320.6ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict2
Detection completed! Check the results in 'runs/detect/predict'.


# **Processing Video**

# **Deploying an Object Detection Model for Live Streaming Using Flask**

In [6]:
video_path = "/content/drive/MyDrive/Objects detection /PXL_20241113_173820763.mp4"
results = model.predict(source=video_path, save=True, conf=0.4)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/931) /content/drive/MyDrive/Objects detection /PXL_20241113_173820763.mp4: 384x640 3 persons, 8 cars, 1 truck, 1 fire hydrant, 276.9ms
video 1/1 (frame 2/931) /content/drive/MyDrive/Objects detection /PXL_20241113_173820763.mp4: 384x640 2 persons, 7 cars, 1 truck, 1 fire hydrant, 241.6ms
video 1/1 (frame 3/931) /content/drive/MyDrive/Objects detection /PXL_20241113_173820763.mp4: 384x640 1 person, 8 cars, 278.4ms
video 1/1 (frame 4/9

In [7]:
pip install flask-ngrok


In [8]:
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip -o ngrok.zip


Archive:  ngrok.zip
  inflating: ngrok                   


# **Objective**

The objective of this assignment is to demonstrate how to deploy a YOLOv8 object detection model in a live-streaming web application using Flask for the backend and OpenCV for video capture and processing. The application will allow real-time object detection through a browser interface.

# **Requirements**

To complete this assignment, ensure the following dependencies are installed in your Python environment:

Python 3.8 or above
Libraries:
Flask: Web framework for the live-streaming application
OpenCV: For capturing and processing video frames
Ultralytics: For loading and running the YOLOv8 model

# **Implementation**

**The implementation involves the following steps:**

**Step 1:**

**Initialize Flask Application**
We will create a simple Flask application to handle the web server and serve the live video feed.

**Step 2: Load YOLOv8 Model**
Using the Ultralytics library, we load a YOLOv8 pretrained or custom-trained model to detect objects in the video feed.

**Step 3: Process Video Feed with OpenCV**
We use OpenCV to capture frames from a webcam (or video source) and process each frame with YOLOv8 for real-time detection.

**Step 4: Stream Video Feed to Browser**
Using Flask’s streaming capabilities, the processed video frames will be sent to a web browser in real time.

In [9]:
# Checking for specific video files in the current directory
import os

# List all files in the current directory
video_files = [f for f in os.listdir('.') if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
video_files

[]

In [ ]:
import os
import uuid
from flask import Flask, request, render_template, Response
from ultralytics import YOLO
import cv2

# Initialize Flask Application
app = Flask(__name__)

# Load the YOLO model
model = YOLO('yolov8n.pt')  # Ensure you have this model in your working directory

# Set up the upload folder
UPLOAD_FOLDER = 'uploads/'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 100 * 1024 * 1024  # Limit uploads to 100MB

# Allowed file extensions
ALLOWED_EXTENSIONS = {'mp4', 'avi', 'mov', 'mkv'}

# Function to check allowed file types
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Function to process the uploaded video
def process_video(video_path):
    video_capture = cv2.VideoCapture(video_path)

    # Handle video capture errors
    if not video_capture.isOpened():
        yield b'Error: Could not open video file.'
        return

    while True:
        success, frame = video_capture.read()
        if not success:
            break

        # YOLO object detection
        results = model.predict(frame, conf=0.5, iou=0.5, verbose=False)

        # Annotate the frame with bounding boxes and labels
        annotated_frame = results[0].plot()

        # Encode the frame as a JPEG image
        _, buffer = cv2.imencode('.jpg', annotated_frame)
        frame_bytes = buffer.tobytes()

        # Yield the frame in a format suitable for browser streaming
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')

    video_capture.release()

# Flask Route: Home Page with Upload Form
@app.route('/')
def index():
    error = request.args.get('error')
    return render_template('index.html', error=error)

# Flask Route: Handle Video Upload
@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return render_template('index.html', error='No file part'), 400

    file = request.files['file']
    if file.filename == '':
        return render_template('index.html', error='No selected file'), 400

    if not allowed_file(file.filename):
        return render_template('index.html', error='Unsupported file type'), 400

    if file:
        # Use a unique filename to prevent overwriting
        unique_filename = f"{uuid.uuid4().hex}_{file.filename}"
        video_path = os.path.join(app.config['UPLOAD_FOLDER'], unique_filename)
        file.save(video_path)

        # Process and stream the video
        return Response(process_video(video_path), mimetype='multipart/x-mixed-replace; boundary=frame')

# Run the Flask App
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
